# [Module 4.1] Train XGBoost

---
## Train with XGBoost on SageMaker Cloud Instance
아 과정은 위의 전처리된 데이타를 가지고 실제 SageMaker Built-in Algorithm XGBoost를 이용하여 학습을 수행 합니다.<br>
실제의 학습 과정은 SageMaker Cloud Instance에서 실제 학습니 됩니다.


Built-in XGboost 알고리즘 컨테이너를 가져옵니다.

In [16]:
!pip install --upgrade sagemaker

     |████████████████████████████████| 294 kB 1.1 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.4.1-py2.py3-none-any.whl size=414442 sha256=9ab36ca37cb0aff5cc307ab8ccffb96b3610761549795505a46c4747ac995950
  Stored in directory: /home/ec2-user/.cache/pip/wheels/19/55/63/76bace7146c2dc97655318d2b37576d5096192f3f03f23dd88
Successfully built sagemaker
  Attempting uninstall: smdebug-rulesconfig
    Found existing installation: smdebug-rulesconfig 0.1.2
    Uninstalling smdebug-rulesconfig-0.1.2:
      Successfully uninstalled smdebug-rulesconfig-0.1.2
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 1.56.1
    Uninstalling sagemaker-1.56.1:
      Successfully uninstalled sagemaker-1.56.1


In [17]:
import sagemaker
import pandas as pd
import numpy as np
import os
import time
import json
from time import strftime, gmtime

In [18]:
role = sagemaker.get_execution_role()

In [19]:
%store -r

In [21]:
# %store

In [29]:
# import boto3

# from sagemaker.amazon.amazon_estimator import get_image_uri
# container = get_image_uri(boto3.Session().region_name, 'xgboost', '1.0-1')




S3에 있는 Train, Validation 전처리된 (Features) 데이타의 경로 및 파일 포맷등을 지정하는 오브젝트를 생성 합니다.

In [30]:
from sagemaker import image_uris, session
xgb_image = image_uris.retrieve("xgboost", session.Session().boto_region_name, version="latest")
print("xgb_image: ", xgb_image)

xgb_image:  306986355934.dkr.ecr.ap-northeast-2.amazonaws.com/xgboost:latest


In [31]:
s3_input_train_processed = sagemaker.session.s3_input(
    preprocessed_pca_train_path, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')
print("S3 Train input: \n")
print(s3_input_train_processed.config)
s3_input_validation_processed = sagemaker.session.s3_input(
    preprocessed_pca_validation_path, 
    distribution='FullyReplicated',
    content_type='text/csv', 
    s3_data_type='S3Prefix')
print("\nS3 Validation input: \n")
print(s3_input_validation_processed.config)

S3 Train input: 

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-ap-northeast-2-057716757052/Scikit-pca-custom/transformtrain-pca-train-output/pca-2020-08-21-03-02-31-990-2020-08-21-03-02-31-990', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'text/csv'}

S3 Validation input: 

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-ap-northeast-2-057716757052/Scikit-pca-custom/transformtrain-pca-validation-output/pca-2020-08-21-03-02-41-179-2020-08-21-03-02-41-180', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'text/csv'}


#### 아래는 약 5분 정도가 소요 됩니다. 아래 셀의 [*] 의 표시가 [숫자] (에: [13])로 바뀔 때까지 기다려 주세요

In [32]:
sess = sagemaker.Session()
instance_type = 'ml.m4.2xlarge'


xgb = sagemaker.estimator.Estimator(xgb_image, # Built-in XGBoost Container
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type= instance_type,
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess
                                   )
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100,
                       )


xgb.fit({'train': s3_input_train_processed, 'validation': s3_input_validation_processed}) 

2020-08-21 05:48:48 Starting - Starting the training job...
2020-08-21 05:48:50 Starting - Launching requested ML instances......
2020-08-21 05:50:12 Starting - Preparing the instances for training......
2020-08-21 05:51:18 Downloading - Downloading input data
2020-08-21 05:51:18 Training - Downloading the training image...
2020-08-21 05:51:43 Uploading - Uploading generated training modelArguments: train
[2020-08-21:05:51:38:INFO] Running standalone xgboost training.
[2020-08-21:05:51:38:INFO] File size need to be processed in the node: 1.46mb. Available memory size in the node: 24474.11mb
[2020-08-21:05:51:38:INFO] Determined delimiter of CSV input is ','
[05:51:38] S3DistributionType set as FullyReplicated
[05:51:38] 2333x25 matrix with 58325 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2020-08-21:05:51:38:INFO] Determined delimiter of CSV input is ','
[05:51:38] S3DistributionType set as FullyReplicated
[05:51:38] 666x25 matrix with 16650 entr

## Inference Pipeline <a class="anchor" id="pipeline_setup"></a>

아래 그림과 같이 위에서 생성한 전처리, 알고리즘 학습, 후처리의 세가지 모델을 가지고 1개의 단일 모델을 만들어 Inference Pipleline을 생성 합니다. <br>
**입력 데이타 가공이 없이 실제 데이타가 입력이 되면, 1개의 단일 모델을 통해서 최종적으로 예측 결과인 True, False의 결과 값이 제공 됩니다.**

![Inference-pipeline](img/Fig2.2.inference_pipeline.png)


**Machine Learning Model Pipeline (Inference Pipeline)는 create_model() 를 호출하여 만들 수 있습니다.** <br>
예를 들어 여기서는 the fitted Scikit-learn inference model, the fitted Xgboost model and the psotprocessing model 의 세가지 모델을 가지고 만듦니다.

아래는 세개 모델을 생성함. 전처리, 후처리 모델 생성시에는 환경 변수를 제공 함

In [33]:
xgb_model_data = xgb.model_data
xgb_image_name = xgb.image_name
print("xgb_model_data: \n", xgb_model_data)
print("xgb_image_name: \n", xgb_image_name)

%store xgb_model_data
%store xgb_image_name

xgb_model_data: 
 s3://sagemaker-ap-northeast-2-057716757052/sagemaker/customer-churn/output/xgboost-2020-08-21-05-48-48-354/output/model.tar.gz
xgb_image_name: 
 306986355934.dkr.ecr.ap-northeast-2.amazonaws.com/xgboost:latest
Stored 'xgb_model_data' (str)
Stored 'xgb_image_name' (str)
